In [1]:
import sqlite3
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
#import streamlit as st
import time
import matplotlib.pyplot as plt
import numpy as np


In [2]:
pip install sqlite3

ERROR: Could not find a version that satisfies the requirement sqlite3 (from versions: none)
ERROR: No matching distribution found for sqlite3
Note: you may need to restart the kernel to use updated packages.


In [2]:


connexion = sqlite3.connect("imdb.db")
cursor = connexion.cursor()



In [3]:


# creating dataframe for movie details table
cursor.execute("""
    SELECT tb.tconst, tb.primaryTitle, tb.genres, tc.directors, tr.averageRating, tr.numVotes
    FROM title_basics tb, title_crew tc, title_ratings tr 
    ON tb.tconst = tc.tconst AND tb.tconst = tr.tconst
    where tb.isAdult = 0 and tb.titleType = 'movie'
""")
data = cursor.fetchall()
col = [description[0] for description in cursor.description]

df_movies = pd.DataFrame.from_records(data=data, columns=col)





In [4]:
cursor.execute("""
    SELECT tp.tconst, tp.nconst
    FROM title_principals tp
    WHERE tp.category = 'actor' or tp.category = 'actress'
""")
data = cursor.fetchall()
col = [description[0] for description in cursor.description]

df_actors = pd.DataFrame.from_records(data=data, columns=col)

In [14]:
groupby = df_actors.groupby('tconst')['nconst'].apply(','.join).reset_index()


df = pd.merge(df_movies, groupby, how='left', on='tconst')
df.shape



(438462, 7)

In [15]:
df.head()

,tconst,primaryTitle,genres,directors,averageRating,numVotes,nconst
0,tt0000009,Miss Jerry,Romance,nm0085156,5.3,200,"nm0063086,nm0183823,nm1309758"
1,tt0000009,Miss Jerry,Romance,nm0085156,5.3,200,"nm0063086,nm0183823,nm1309758"
2,tt0000502,Bohemios,\N,nm0063413,4.2,14,"nm0215752,nm0252720"
3,tt0000502,Bohemios,\N,nm0063413,4.2,14,"nm0215752,nm0252720"
4,tt0000574,The Story of the Kelly Gang,"Action,Adventure,Biography",nm0846879,6.0,797,"nm0846887,nm0846894,nm1431224,nm3002376"


In [16]:


df = df.drop('tconst', axis=1)
df = df.rename(columns={"nconst": "actors"})



In [17]:


df['averageRating'] = df['averageRating'].astype(float)
df['numVotes'] = df['numVotes'].astype(int)



In [30]:
df_clean=df
df_clean

,primaryTitle,genres,directors,averageRating,numVotes,actors
0,Miss Jerry,Romance,nm0085156,5.3,200,"nm0063086,nm0183823,nm1309758"
1,Miss Jerry,Romance,nm0085156,5.3,200,"nm0063086,nm0183823,nm1309758"
2,Bohemios,\N,nm0063413,4.2,14,"nm0215752,nm0252720"
3,Bohemios,\N,nm0063413,4.2,14,"nm0215752,nm0252720"
4,The Story of the Kelly Gang,"Action,Adventure,Biography",nm0846879,6.0,797,"nm0846887,nm0846894,nm1431224,nm3002376"
...,...,...,...,...,...,...
438457,Manoharam,"Comedy,Drama",nm7011994,6.8,831,"nm2068971,nm10375007,nm7243877,nm1428724"
438458,Footloose in the Cotswolds: Part 1,Documentary,nm1645815,7.0,10,nm1644256
438459,Footloose in the Cotswolds: Part 1,Documentary,nm1645815,7.0,10,nm1644256
438460,The Cursed,"Fantasy,Horror,Mystery",nm1193346,6.2,11360,"nm2933542,nm0717709,nm0677944,nm3646923"


In [31]:


# convert data types
df_clean['genres'] = df_clean['genres'].astype("string")
df_clean['directors'] = df_clean['directors'].astype("string")
df_clean['actors'] = df_clean['actors'].astype("string")



In [19]:


df_clean.dtypes



primaryTitle      object
genres            string
directors         string
averageRating    float64
numVotes           int64
actors            string
dtype: object

In [32]:
df_clean

,primaryTitle,genres,directors,averageRating,numVotes,actors
0,Miss Jerry,Romance,nm0085156,5.3,200,"nm0063086,nm0183823,nm1309758"
1,Miss Jerry,Romance,nm0085156,5.3,200,"nm0063086,nm0183823,nm1309758"
2,Bohemios,\N,nm0063413,4.2,14,"nm0215752,nm0252720"
3,Bohemios,\N,nm0063413,4.2,14,"nm0215752,nm0252720"
4,The Story of the Kelly Gang,"Action,Adventure,Biography",nm0846879,6.0,797,"nm0846887,nm0846894,nm1431224,nm3002376"
...,...,...,...,...,...,...
438457,Manoharam,"Comedy,Drama",nm7011994,6.8,831,"nm2068971,nm10375007,nm7243877,nm1428724"
438458,Footloose in the Cotswolds: Part 1,Documentary,nm1645815,7.0,10,nm1644256
438459,Footloose in the Cotswolds: Part 1,Documentary,nm1645815,7.0,10,nm1644256
438460,The Cursed,"Fantasy,Horror,Mystery",nm1193346,6.2,11360,"nm2933542,nm0717709,nm0677944,nm3646923"


In [33]:


# Function to convert all strings to lower case and strip names of spaces
def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        #Check if director exists. If not, return empty string
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''



In [34]:




# Apply clean_data function to your features.
features = ['directors', 'genres', 'actors']

for feature in features:
    df_clean[feature] = df_clean[feature].apply(clean_data)





In [35]:



def create_soup(x):
    return ' '.join(x['directors']) + ' ,' + ' '.join(x['genres']) + ' ,' + ' '.join(x['actors'])





In [36]:


# Create a new soup column representing the features
df_clean['soup'] = df_clean.apply(create_soup, axis=1)



In [37]:


# clean the soup column
features = ['soup']

for feature in features:
    df_clean[feature] = df_clean[feature].apply(clean_data)



In [38]:


print(df_clean['soup'])



0           nm0085156,romance,nm0063086,nm0183823,nm1309758
1           nm0085156,romance,nm0063086,nm0183823,nm1309758
2                          nm0063413,\n,nm0215752,nm0252720
3                          nm0063413,\n,nm0215752,nm0252720
4         nm0846879,action,adventure,biography,nm0846887...
                                ...                        
438457    nm7011994,comedy,drama,nm2068971,nm10375007,nm...
438458                      nm1645815,documentary,nm1644256
438459                      nm1645815,documentary,nm1644256
438460    nm1193346,fantasy,horror,mystery,nm2933542,nm0...
438461    nm1193346,fantasy,horror,mystery,nm2933542,nm0...
Name: soup, Length: 438462, dtype: object


In [39]:
# Function to convert all strings to lower case and strip names of spaces
df_clean['soup'] = df_clean['soup'].replace(',', ' ', regex=True)

In [40]:


df_clean.head()



,primaryTitle,genres,directors,averageRating,numVotes,actors,soup
0,Miss Jerry,romance,nm0085156,5.3,200,"nm0063086,nm0183823,nm1309758",nm0085156 romance nm0063086 nm0183823 nm1309758
1,Miss Jerry,romance,nm0085156,5.3,200,"nm0063086,nm0183823,nm1309758",nm0085156 romance nm0063086 nm0183823 nm1309758
2,Bohemios,\n,nm0063413,4.2,14,"nm0215752,nm0252720",nm0063413 \n nm0215752 nm0252720
3,Bohemios,\n,nm0063413,4.2,14,"nm0215752,nm0252720",nm0063413 \n nm0215752 nm0252720
4,The Story of the Kelly Gang,"action,adventure,biography",nm0846879,6.0,797,"nm0846887,nm0846894,nm1431224,nm3002376",nm0846879 action adventure biography nm0846887...


In [41]:
df_clean['soup'] = df_clean['soup'].replace(',', ' ', regex=True)


In [42]:


df_clean['primaryTitle'] = df_clean['primaryTitle'].astype("string")



In [43]:


df_clean = df_clean.reset_index()



In [44]:
df_clean.head(300)

,index,primaryTitle,genres,directors,averageRating,numVotes,actors,soup
0,0,Miss Jerry,romance,nm0085156,5.3,200,"nm0063086,nm0183823,nm1309758",nm0085156 romance nm0063086 nm0183823 nm1309758
1,1,Miss Jerry,romance,nm0085156,5.3,200,"nm0063086,nm0183823,nm1309758",nm0085156 romance nm0063086 nm0183823 nm1309758
2,2,Bohemios,\n,nm0063413,4.2,14,"nm0215752,nm0252720",nm0063413 \n nm0215752 nm0252720
3,3,Bohemios,\n,nm0063413,4.2,14,"nm0215752,nm0252720",nm0063413 \n nm0215752 nm0252720
4,4,The Story of the Kelly Gang,"action,adventure,biography",nm0846879,6.0,797,"nm0846887,nm0846894,nm1431224,nm3002376",nm0846879 action adventure biography nm0846887...
...,...,...,...,...,...,...,...,...
295,295,Bánk bán,\n,nm0002031,5.1,29,"nm1110269,nm0433425,nm12016940,nm12016941",nm0002031 \n nm1110269 nm0433425 nm12016940 nm...
296,296,Cabiria,"adventure,drama,history",nm0665163,7.1,3629,"nm0021935,nm0702894,nm0656034,nm0146028",nm0665163 adventure drama history nm0021935 nm...
297,297,Cabiria,"adventure,drama,history",nm0665163,7.1,3629,"nm0021935,nm0702894,nm0656034,nm0146028",nm0665163 adventure drama history nm0021935 nm...
298,298,The Call of the North,"adventure,drama","nm0002177,nm0001124",5.2,45,"nm0114307,nm0249187,nm0731623,nm0455612,nm0139...",nm0002177 nm0001124 adventure drama nm0114307 ...


In [114]:
df_clean[df_clean['primaryTitle']=='Titanic']

,index,primaryTitle,startYear,genres,directors,averageRating,numVotes,soup


In [45]:
df_clean.to_csv('df_clean_bigger.csv')

In [46]:


# Import CountVectorizer and create the count matrix
count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(df_clean['soup'])



In [47]:


count_matrix.shape
print(count_matrix)



  (0, 17725)	1
  (0, 427713)	1
  (0, 14085)	1
  (0, 33131)	1
  (0, 198765)	1
  (1, 17725)	1
  (1, 427713)	1
  (1, 14085)	1
  (1, 33131)	1
  (1, 198765)	1
  (2, 14143)	1
  (2, 37740)	1
  (2, 43206)	1
  (3, 14143)	1
  (3, 37740)	1
  (3, 43206)	1
  (4, 126404)	1
  (4, 0)	1
  (4, 2)	1
  (4, 4)	1
  (4, 126405)	1
  (4, 126406)	1
  (4, 213377)	1
  (4, 286350)	1
  (5, 126404)	1
  :	:
  (438457, 390963)	1
  (438457, 151079)	1
  (438457, 387836)	1
  (438458, 7)	1
  (438458, 225419)	1
  (438458, 225281)	1
  (438459, 7)	1
  (438459, 225419)	1
  (438459, 225281)	1
  (438460, 10)	1
  (438460, 18)	1
  (438460, 15)	1
  (438460, 109459)	1
  (438460, 283810)	1
  (438460, 180129)	1
  (438460, 104025)	1
  (438460, 310557)	1
  (438461, 10)	1
  (438461, 18)	1
  (438461, 15)	1
  (438461, 109459)	1
  (438461, 283810)	1
  (438461, 180129)	1
  (438461, 104025)	1
  (438461, 310557)	1


In [48]:
# Function that takes in a list of movie titles as input and outputs most similar movies ON THE FLY
def get_recommendations_list(liked_movies_list, similar_movies=10, vectorized_count_matrix=count_matrix):
    # Get the index of the movies that matches the title
    movie_id_list=[]
    for movie_title in liked_movies_list:
        movie_id = df_clean.index[df_clean['primaryTitle'] == movie_title]
        movie_id_list.append(movie_id[0])

    df_sim_list=pd.DataFrame()
    print(movie_id_list)
    for movie_id in movie_id_list:
        vector = vectorized_count_matrix[movie_id]
        simil = vector.dot(count_matrix.T).todense()[0]
        sim_scores = np.squeeze(np.array(simil))
        df_sim=pd.DataFrame(sim_scores.reshape(-1),columns=[movie_id])
        df_sim_list[movie_id]=df_sim

    df_sim_list['average']=df_sim_list.mean(numeric_only=True, axis=1)

    rslt_df = df_sim_list.sort_values(by='average',ascending=False)

    # print(rslt_df[0:11])
    rec_list=rslt_df.reset_index()
    list1=[]
    list1=rec_list['index'][len(liked_movies_list):(len(liked_movies_list)+similar_movies)].values

    # Return the top 10 most similar movies
    return df_clean[['primaryTitle', 'genres']].iloc[list1]

In [49]:


liked_movies_list=['The Little Mermaid','The Lion King', 'Pocahontas']

recommendations = get_recommendations_list(liked_movies_list)
print(recommendations)



[25002, 64838, 68482]
                                         primaryTitle  \
68482                                      Pocahontas   
25003                              The Little Mermaid   
25002                              The Little Mermaid   
266845  Gekijô ban Gegege no Kitarô: Nippon bakuretsu   
430799                       Conan, the Boy in Future   
430798                       Conan, the Boy in Future   
434506                              Nobody's Boy Remi   
266844  Gekijô ban Gegege no Kitarô: Nippon bakuretsu   
317313                             Secret of the Seal   
75165                             The Prince of Egypt   

                           genres  
68482   adventure,animation,drama  
25003        drama,family,fantasy  
25002        drama,family,fantasy  
266845  adventure,animation,drama  
430799  adventure,animation,drama  
430798  adventure,animation,drama  
434506  adventure,animation,drama  
266844  adventure,animation,drama  
317313  adventure,animatio